# Cross Validation, Model Evaluation and Selection

## Part 1

### Cross Validation

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,  cross_val_score, StratifiedKFold, KFold, RepeatedKFold, GridSearchCV
from sklearn.datasets import load_digits, load_breast_cancer, make_classification, make_blobs
 
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, roc_auc_score
from sklearn.dummy import DummyRegressor
from matplotlib import pyplot
from sklearn.svm import SVC
from sklearn.metrics import SCORERS 
from matplotlib.colors import ListedColormap

In [ ]:
fruits = pd.read_csv('datasets/fruit_data_with_colors.txt', sep='\t')
fruits.head()
feature_names_fruits = ['height', 'width', 'mass', 'color_score']

X_fruits = fruits[feature_names_fruits]
y_fruits = fruits['fruit_label']

X_fruits_2d = fruits[['height', 'width']]
y_fruits_2d = fruits['fruit_label']

In [ ]:
X_fruits_2d

In [ ]:
### Example based on k-NN classifier with fruit dataset (2 features)

from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 5)
X = X_fruits_2d.to_numpy() # we will check the difference in the following. 
y = y_fruits_2d.to_numpy()


In [ ]:
y

In [ ]:
#cv_scores = cross_val_score(clf, X, y, cv=5)

cv_fix = 5

kf5 = KFold(n_splits=5, shuffle=True)
#Shuffle makes a random split, Otherwise it splits the data into 5 folds without shuffling.

cv5 = RepeatedKFold(n_splits=5, n_repeats=12) 
#Repeats K-Fold n times with different randomization in each repetition.

 

skf = StratifiedKFold(n_splits=2, shuffle=True)
#This cross-validation object is a variation of KFold that returns stratified folds. 
#The folds are made by preserving the percentage of samples for each class.

rskf = RepeatedStratifiedKFold(n_splits=3, n_repeats=8)
#Repeats stratified K-Fold n times with different randomization in each repetition.

CVResult = cross_val_score(clf, X, y, cv=rskf)
print('Cross-validation scores:', CVResult)
print('Mean cross-validation score : {:.3f}'.format(np.mean(CVResult)))

# kf5 is 5 fold CV with random shuffling. Please retry it several times 
# to see very awkward individual results
# However the average is not changing too much...

### A note on performing cross-validation for more advanced scenarios.

In some cases (e.g. when feature values have very different ranges), we've seen the need to scale or normalize the training and test sets before use with a classifier. The proper way to do cross-validation when you need to scale the data is *not* to scale the entire dataset with a single transform, since this will indirectly leak information into the training data about the whole dataset, including the test data (see the lecture on data leakage later in the course).  Instead, scaling/normalizing must be computed and applied for each cross-validation fold separately.  To do this, the easiest way in scikit-learn is to use *pipelines*.  While these are beyond the scope of this course, further information is available in the scikit-learn documentation here:

http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

or the Pipeline section in the recommended textbook: Introduction to Machine Learning with Python by Andreas C. Müller and Sarah Guido (O'Reilly Media).

## Evaluation for Classification


### Data set for handwritten images. 

In [ ]:


## This is a dataset which has 64 features and the results are the handwritten images for numbers
dataset = load_digits()
X, y = dataset.data, dataset.target



In [ ]:
X.shape

In [ ]:
list(zip(dataset.target_names, np.bincount(dataset.target)))

In [ ]:
#rewrite the same thing.  
for class_name, class_count in zip(dataset.target_names, np.bincount(dataset.target)):
    print(class_name,class_count)

In [ ]:
pd.DataFrame(X).head(11)

In [ ]:
pd.DataFrame(y).head(22)

In [ ]:
# Creating a dataset with imbalanced binary classes:  
# Negative class (0) is 'not digit 1' 
# Positive class (1) is 'digit 1'
y_binary_imbalanced = y.copy()
y_binary_imbalanced[y_binary_imbalanced != 1] = 0


In [ ]:
np.bincount(y_binary_imbalanced) #only two classes here. 

In [ ]:
182/(1615+182)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)

# Accuracy of Support Vector Machine classifier
from sklearn.svm import SVC

svm = SVC(kernel='rbf', C=1).fit(X_train, y_train)
svm.score(X_test, y_test)

In [ ]:
from sklearn.dummy import DummyClassifier
#produces predictions based on the most frequent class. 
# That is it assigns the value of the most frequent class to every body. 
# Therefore the dummy 'most_frequent' classifier always predicts class 0


# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
y_dummy_predictions = dummy_majority.predict(X_test)

y_dummy_predictions

In [ ]:
dummy_majority.score(X_test, y_test)

In [ ]:
svm = SVC(kernel='linear', C=1).fit(X_train, y_train)
svm.score(X_test, y_test)

### Confusion matrices


#### Binary (two-class) confusion matrix


#### confusion matrix for the constant dummy classifies


In [ ]:
#produces predictions based on the most frequent class. 
# That is it assigns the value of the most frequent class to every body. 

#confusion matrix for the constant dummy classifies


from sklearn.metrics import confusion_matrix

# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
y_majority_predicted = dummy_majority.predict(X_test)
confusion = confusion_matrix(y_test, y_majority_predicted)

print('Most frequent class (dummy classifier)\n', confusion)

#### confusion matrix for the stratified dummy classifies


In [ ]:


# produces random predictions w/ same class proportion as training set


dummy_classprop = DummyClassifier(strategy='stratified').fit(X_train, y_train)
y_classprop_predicted = dummy_classprop.predict(X_test)
confusion = confusion_matrix(y_test, y_classprop_predicted)

print('Random class-proportional prediction (dummy classifier)\n', confusion)

In [ ]:
5400/450

#### #confusion matrix for linear SVM

In [ ]:
svm = SVC(kernel='linear', C=1).fit(X_train, y_train)
svm_predicted = svm.predict(X_test)
confusion = confusion_matrix(y_test, svm_predicted)

print('Support vector machine classifier (linear kernel, C=1)\n', confusion)

#### confusion matrix for logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression().fit(X_train, y_train) 
# Note that we define the logistic regression as lr above
lr_predicted = lr.predict(X_test)
confusion = confusion_matrix(y_test, lr_predicted)

print('Logistic regression classifier (default settings)\n', confusion)

### Evaluation metrics for binary classification


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report


# Accuracy = TP + TN / (TP + TN + FP + FN)
# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)  Also known as sensitivity, or True Positive Rate
# F1 = 2 * Precision * Recall / (Precision + Recall) 
'''
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, tree_predicted)))
print('Precision: {:.2f}'.format(precision_score(y_test, tree_predicted)))
print('Recall: {:.2f}'.format(recall_score(y_test, tree_predicted)))
print('F1: {:.2f}'.format(f1_score(y_test, tree_predicted)))

# Combined report with all above metrics

print(classification_report(y_test, tree_predicted, target_names=['not 1', '1']))

'''

In [ ]:
print('Random class-proportional (dummy)\n', 
      classification_report(y_test, y_classprop_predicted, target_names=['not 1', '1'])) 
print('SVM\n', 
      classification_report(y_test, svm_predicted, target_names = ['not 1', '1']))
print('Logistic regression\n', 
      classification_report(y_test, lr_predicted, target_names = ['not 1', '1']))
#print('Decision tree\n', 
#      classification_report(y_test, tree_predicted, target_names = ['not 1', '1']))

## Part 2

### Decision functions 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)
y_scores_lr = lr.fit(X_train, y_train).decision_function(X_test) 
# recall that we have defined the logistic regression as lr above using the following
# lr = LogisticRegression().fit(X_train, y_train) 

# The decision function tells us on which side of the hyperplane generated by the classifier
# we are (and how far we are away from it). 
# Based on that information, the estimator then label the examples with the corresponding label.


y_scores_lr

In [ ]:
#logistic regression
y_score_list = list(zip(y_test[0:20], y_scores_lr[0:20]))
#prediction function is (b + w1x1 + w2x2 + ...)
# show the decision_function scores for first 20 instances
y_score_list

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)
y_proba_lr = lr.fit(X_train, y_train).predict_proba(X_test)
y_proba_list = list(zip(y_test[0:50], y_proba_lr[0:50,1]))
# predict proba is h_theta()
# show the probability of positive class for first 20 instances
# given by 1/(1+exp(-z)) where z is given above by the decision function. 
y_proba_list

### Precision-recall curves

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, y_scores_lr)
# python automatically determines precision and recall values for different threshold values. 
# recall that y_test are the real y values of the test set
# y_scores_lr are the distances from the decision boundary   




In [ ]:
# pick the threshold value which is closest to zero
# Why? 
# recall that our default threshold value is zero
# hence we will plot it to see on the graph. 
closest_zero = np.argmin(np.abs(thresholds)) # 


# find the corresponding precision and recall values 
closest_zero_p = precision[closest_zero]
closest_zero_r = recall[closest_zero]



In [ ]:
thresholds[closest_zero]

In [ ]:
plt.figure()
plt.xlim([0.0, 1.01])
plt.ylim([0.0, 1.01])
plt.plot(precision, recall, label='Precision-Recall Curve')
plt.plot(closest_zero_p, closest_zero_r, 'o', markersize = 12, fillstyle = 'none', c='r', mew=3)


### ROC curves, Area-Under-Curve (AUC)

In [ ]:
from sklearn.metrics import roc_curve, auc

X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)
#lr = LogisticRegression(C=0.000001).fit(X_train, y_train)
y_score_lr = lr.fit(X_train, y_train).decision_function(X_test)
#recall the decision function=> gives the distance to the decision boundary


In [ ]:
y_score_lr 

In [ ]:
# calculate the false positive rate and true positive rate

fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)

# find the area under the curve
roc_auc_lr = auc(fpr_lr, tpr_lr)


In [ ]:
roc_auc_lr 

In [ ]:
plt.figure()
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_lr, tpr_lr, lw=3, label='LogRegr ROC curve (area = {:0.2f})'.format(roc_auc_lr))
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.show()

In [ ]:
tpr_lr

In [ ]:
from matplotlib import cm

X_train, X_test, y_train, y_test = train_test_split(X, y_binary_imbalanced, random_state=0)

plt.figure()
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
for g in [0.01, 0.1, 0.20, 1]:
    svm = SVC(gamma=g).fit(X_train, y_train)
    y_score_svm = svm.decision_function(X_test)
    fpr_svm, tpr_svm, _ = roc_curve(y_test, y_score_svm)
    roc_auc_svm = auc(fpr_svm, tpr_svm)
    accuracy_svm = svm.score(X_test, y_test)
    print("gamma = {:.2f}  accuracy = {:.2f}   AUC = {:.2f}".format(g, accuracy_svm, 
                                                                    roc_auc_svm))
    plt.plot(fpr_svm, tpr_svm, lw=3, alpha=0.7, 
             label='SVM (gamma = {:0.2f}, area = {:0.2f})'.format(g, roc_auc_svm))

plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate (Recall)', fontsize=16)
plt.plot([0, 1], [0, 1], color='k', lw=0.5, linestyle='--')
plt.legend(loc="lower right", fontsize=11)
plt.title('ROC curve: (1-of-10 digits classifier)', fontsize=16)
#plt.axes().set_aspect('equal')

plt.show()

In [ ]:
fpr_svm

### Evaluation measures for multi-class classification (at home)


#### Multi-class confusion matrix

In [ ]:
dataset = load_digits()
X, y = dataset.data, dataset.target
X_train_mc, X_test_mc, y_train_mc, y_test_mc = train_test_split(X, y, random_state=0)


svm = SVC(kernel = 'linear').fit(X_train_mc, y_train_mc)
svm_predicted_mc = svm.predict(X_test_mc)
confusion_mc = confusion_matrix(y_test_mc, svm_predicted_mc)
df_cm = pd.DataFrame(confusion_mc, 
                     index = [i for i in range(0,10)], columns = [i for i in range(0,10)])

plt.figure(figsize=(5.5,4))
sns.heatmap(df_cm, annot=True)
plt.title('SVM Linear Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(y_test_mc, 
                                                                       svm_predicted_mc)))
plt.ylabel('True label')
plt.xlabel('Predicted label')


svm = SVC(kernel = 'rbf').fit(X_train_mc, y_train_mc)
svm_predicted_mc = svm.predict(X_test_mc)
confusion_mc = confusion_matrix(y_test_mc, svm_predicted_mc)
df_cm = pd.DataFrame(confusion_mc, index = [i for i in range(0,10)],
                  columns = [i for i in range(0,10)])

plt.figure(figsize = (5.5,4))
sns.heatmap(df_cm, annot=True)
plt.title('SVM RBF Kernel \nAccuracy:{0:.3f}'.format(accuracy_score(y_test_mc, 
                                                                    svm_predicted_mc)))
plt.ylabel('True label')
plt.xlabel('Predicted label');

#### Multi-class classification report

In [ ]:
print(classification_report(y_test_mc, svm_predicted_mc))

#### Micro- vs. macro-averaged metrics

In [ ]:
print('Micro-averaged precision = {:.2f} (treat instances equally)'
      .format(precision_score(y_test_mc, svm_predicted_mc, average = 'micro')))
print('Macro-averaged precision = {:.2f} (treat classes equally)'
      .format(precision_score(y_test_mc, svm_predicted_mc, average = 'macro')))

In [ ]:
print('Micro-averaged f1 = {:.2f} (treat instances equally)'
      .format(f1_score(y_test_mc, svm_predicted_mc, average = 'micro')))
print('Macro-averaged f1 = {:.2f} (treat classes equally)'
      .format(f1_score(y_test_mc, svm_predicted_mc, average = 'macro')))


#  Regression Evaluation Metrics


In [ ]:
house_price = pd.DataFrame (np.genfromtxt("house_price_data_2000.csv", \
            delimiter=";", skip_header=1, dtype=None))
house_price.head()

X_house = house_price.iloc[:  , 0:1] #from 0(inclusive) to 3(exclusive)
y_house = house_price.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X_house, y_house, random_state=0)


In [ ]:

lm = LinearRegression().fit(X_train, y_train)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train, y_train)

y_predict = lm.predict(X_test)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test)

print('Linear model, coefficients: ', lm.coef_)
print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(y_test, 
                                                                     y_predict_dummy_mean)))
print("Mean squared error (linear model): {:.2f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score (dummy): {:.2f}".format(r2_score(y_test, y_predict_dummy_mean)))
print("r2_score (linear model): {:.2f}".format(r2_score(y_test, y_predict)))

# Plot outputs
plt.scatter(X_test, y_test,  color='black')
#plt.plot(X_test[0], y_test, color='green', linewidth=2)
plt.plot(X_test[0], y_predict, color='green', linewidth=2)
plt.plot(X_test[0], y_predict_dummy_mean, color='red', linestyle = 'dashed', 
         linewidth=2, label = 'dummy')

plt.show()


#### Use another dataset (diabetes dataset) from Scikit for the dummy regressor

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor

diabetes = datasets.load_diabetes()

X = diabetes.data[:, None, 6]
y = diabetes.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

lm = LinearRegression().fit(X_train, y_train)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(X_train, y_train)

y_predict = lm.predict(X_test)
y_predict_dummy_mean = lm_dummy_mean.predict(X_test)



In [ ]:
print('Linear model, coefficients: ', lm.coef_)
print("Mean squared error (dummy): {:.2f}".
      format(mean_squared_error(y_test,y_predict_dummy_mean)))
print("Mean squared error (linear model): {:.2f}".format(mean_squared_error(y_test, y_predict)))
print("r2_score (dummy): {:.2f}".format(r2_score(y_test, y_predict_dummy_mean)))
print("r2_score (linear model): {:.2f}".format(r2_score(y_test, y_predict)))



In [ ]:
# Plot outputs
plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, y_predict, color='green', linewidth=2)
plt.plot(X_test, y_predict_dummy_mean, color='red', linestyle = 'dashed', 
         linewidth=2, label = 'dummy')

plt.show()

# Model selection using evaluation metrics

We will discuss ways of selecting models using these evaluation metrics. So far, we did it in our assignments but now we will discuss it in more details and use a structural way through scikit learn library

## Cross-validation

You can use cross validation with different metrics...

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

dataset = load_digits()
# again, making this a binary problem with 'digit 1' as positive class 
# and 'not 1' as negative class
X, y = dataset.data, dataset.target == 1
clf = SVC(kernel='linear', C=1)# ==> it's not executed yet. 

# accuracy is the default scoring metric
print('Cross-validation (accuracy)', cross_val_score(clf, X, y, cv=5))
# use AUC as scoring metric
print('Cross-validation (AUC)', cross_val_score(clf, X, y, cv=5, scoring = 'roc_auc'))
# use recall as scoring metric
print('Cross-validation (recall)', cross_val_score(clf, X, y, cv=5, scoring = 'recall'))

## Grid search example

Grid search is an exhaustive search over specified parameter values for an estimator. 

We will use grid search to find the the optimal parameters instead of manually searching them.  


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import SCORERS

dataset = load_digits()
X, y = dataset.data, dataset.target == 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf = SVC(kernel='rbf') # again, just like we did previously, we define a classifier object. 
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100]}


# Note that previously we would call the fit function of this clf object.
# But now we  give this object to the GridSearch and then run the fit function
# of the gridsearch to find the optimal parameters over different gamma values. 

grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)

# note that default metric to optimize over grid parameters: accuracy
#### What are the other evaluation metrics supported for model selection
grid_clf_acc

In [ ]:
y

 #### What are the other evaluation metrics supported for model selection

In [ ]:
from sklearn.metrics import SCORERS

print(sorted(list(SCORERS.keys())))

In [ ]:

#Now we are calling the fit function of the grid search to fit the function thru different gamma
grid_clf_acc.fit(X_train, y_train)

#fit the y values using the final decision function of the grid search

#print out the best parameter and the best score
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_ )
print('Grid best score (accuracy): ', grid_clf_acc.best_score_ )


In [ ]:
y_train

In [ ]:
# alternative metric to optimize over grid parameters: AUC
grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
grid_clf_auc.fit(X_train, y_train)


print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
print('Grid best score (AUC): ', grid_clf_auc.best_score_)

In [ ]:
y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test) 
print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))

### You can also run the grid search over multiple parameters. 

In [ ]:
clf = SVC()
grid_values = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
#svc = svm.SVC()

grid_clf_acc = GridSearchCV(clf, grid_values)
grid_clf_acc.fit(X_train, y_train)

#print out the best parameter and the best score
print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_ )
print('Grid best score (accuracy): ', grid_clf_acc.best_score_ )